In [99]:
# @title SET CONFIGURATIONS AND CREATE YOUR FANTASY TEAM
import pandas as pd
import numpy as np
import pulp as plp
import ipywidgets as widgets
from IPython.display import display
class FantasyTeamGenerator:
    def __init__(self, config_params = {}, with_widgets=False, output_folder=None, print_result = False):

        # Initialization
        if print_result:
            self.initialize_display_options()

        self.initialize_variables(config_params)
        self.print_result = print_result
        self.output_folder = output_folder
        if with_widgets:
            # Initialize widgets
            self.initialize_team_widgets()

            # Observe widgets
            self.observe_team_widgets()

            # Display widgets
            self.view_team_widgets()

    def initialize_display_options(self):

        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)
        pd.set_option('display.max_colwidth', -1)

    def initialize_variables(self, params):
        self.data = pd.read_csv(
            'https://raw.githubusercontent.com/Lakshmi-1212/FantasyTeamGenerator/master/input/player_details.csv')
        
        # If line up is out, consider players in playing 11, else take players who have played the previous game
        self.CONSIDER_PLAYERS = params.get('CONSIDER_PLAYERS', 'ALL')  # ALL, PLAYED_LAST_MATCH, IN_PLAYING_11, PLAYED_ATLEAST_ONE_MATCH

        self.TEAM1 = params.get('TEAM1', 'CSK')
        self.TEAM2 = params.get('TEAM2', 'MI')

        self.MIN_PLAYERS_TEAM1 = params.get('MIN_PLAYERS_TEAM1', 4)  # Range 4 to 7
        self.MAX_PLAYERS_TEAM1 = params.get('MAX_PLAYERS_TEAM1', 7)  # Range 4 to 7

        self.OPTIMIZE_ON = params.get('OPTIMIZE_ON',
                                      'POINTS')  # optimize on POINTS/POPULARITY/COUNTDREAMTEAM/AVGPOINTSPERMATCH/COUNTDREAMTEAMPERCENT

        self.CONSIDER_TOP_N_BATSMEN = params.get('CONSIDER_TOP_N_BATSMEN',
                                                 5)  # Consider only batsmen who bat in top n for BAT - Keep 11 to consider all players

        # self.GROUND = 'SHARJAH'  # DUBAI/ABUDHABI/SHARJAH - TODO: check how to use this?

        self.MAX_BOWLERS = params.get('MAX_BOWLERS', 6)  # Between 3 to 6. Choose 3 to have more batsmen in the team (SHARJAH?)
        self.MAX_BATSMEN = params.get('MAX_BATSMEN', 6)  # Between 3 to 6.
        self.MAX_AR = params.get('MAX_AR', 4)  # Between 1 to 4.
        self.MAX_WK = params.get('MAX_WK', 4)  # Between 1 to 4.

        self.SPIN_WEIGHTAGE_FACTOR = params.get('SPIN_WEIGHTAGE_FACTOR',
                                                1)  # To give more weightage to spinners (DUBAI/ABU DHABI). No weightage - Keep as 1 if no preference has to be given for spinners.

        self.MIN_CREDITS = params.get('MIN_CREDITS', 98)  # Max credits that can be utilized is 100

        self.MUST_HAVE_IN_TEAM = params.get('MUST_HAVE_IN_TEAM', [])
        self.EXCLUDE_FROM_TEAM = params.get('EXCLUDE_FROM_TEAM', [])
        self.update_after_team_selection()
        self.max_players_list = [count for count in range(self.MIN_PLAYERS_TEAM1, 7 + 1)]
        self.show_team = False

    ################## WIDGET CONFIGURATIONS - BEGIN ########################
    def initialize_team_widgets(self):

        self.w_lineup = widgets.Dropdown(options=['ALL', 'PLAYED_LAST_MATCH', 'IN_PLAYING_11', 'PLAYED_ATLEAST_ONE_MATCH'],
                                         value=self.CONSIDER_PLAYERS, description='Consider Players:',
                                         style={'description_width': 'initial'}, disabled=False)

        self.w_team1 = widgets.Dropdown(options=self.data['Team'].unique(), description='Select Team 1:',
                                        value=self.TEAM1,
                                        style={'description_width': 'initial'}, disabled=False)

        self.w_team2 = widgets.Dropdown(options=self.data['Team'].unique(), description='Select Team 2:',
                                        value=self.TEAM2,
                                        style={'description_width': 'initial'}, disabled=False)

        self.w_optimize = widgets.Dropdown(
            options=['POINTS', 'POPULARITY', 'DREAM TEAM COUNT', 'AVG POINTS PER MATCH', 'DREAM TEAM COUNT %'],
            value='POINTS',
            description='Select team on:',
            style={'description_width': 'initial'}, disabled=False)

        self.w_min_team1 = widgets.Dropdown(options=[4, 5, 6, 7], value=self.MIN_PLAYERS_TEAM1,
                                            description='Minimum players:',
                                            style={'description_width': 'initial'}, disabled=False)

        self.w_max_team1 = widgets.Dropdown(options=self.max_players_list, value=self.MAX_PLAYERS_TEAM1,
                                            description='Maximum players:',
                                            style={'description_width': 'initial'}, disabled=False)

        self.w_top_n_bat = widgets.Dropdown(options=[count for count in range(1, 11 + 1)],
                                            value=self.CONSIDER_TOP_N_BATSMEN,
                                            description='Consider Top N Batsmen :',
                                            style={'description_width': 'initial'}, disabled=False)

        self.w_max_bowlers = widgets.Dropdown(options=[count for count in range(3, 6 + 1)], value=self.MAX_BOWLERS,
                                              description='Maximum bowlers in team:',
                                              style={'description_width': 'initial'}, disabled=False)

        self.w_max_batsmen = widgets.Dropdown(options=[count for count in range(3, 6 + 1)], value=self.MAX_BATSMEN,
                                              description='Maximum batsmen in team:',
                                              style={'description_width': 'initial'}, disabled=False)

        self.w_max_wk = widgets.Dropdown(options=[count for count in range(1, 4 + 1)], value=self.MAX_WK,
                                         description='Maximum wicket-keepers in team:',
                                         style={'description_width': 'initial'}, disabled=False)

        self.w_max_ar = widgets.Dropdown(options=[count for count in range(1, 4 + 1)], value=self.MAX_AR,
                                         description='Maximum all-rounders in team:',
                                         style={'description_width': 'initial'}, disabled=False)

        self.w_spin_weightage = widgets.Dropdown(options=[0.5, 0.75, 1, 1.25, 1.5, 1.75, 2],
                                                 value=self.SPIN_WEIGHTAGE_FACTOR,
                                                 description='Weightage for spinners :',
                                                 style={'description_width': 'initial'}, disabled=False)

        self.w_min_credits = widgets.FloatSlider(value=self.MIN_CREDITS, min=90, max=100.0, step=0.5,
                                                 description='Min credits value:',
                                                 disabled=False, continuous_update=False, orientation='horizontal',
                                                 readout=True, readout_format='.1f',
                                                 style={'description_width': 'initial'})
        self.multiselect_layout = widgets.Layout(width='40%')
        self.w_musthave_players = widgets.SelectMultiple(description='Must have players in team:',
                                                         style={'description_width': 'initial'},
                                                         options=self.players_list,
                                                         layout=self.multiselect_layout)
        self.w_exclude_players = widgets.SelectMultiple(description='Exclude players in team:',
                                                         style={'description_width': 'initial'},
                                                         options=self.players_list,
                                                         layout=self.multiselect_layout)



        button_layout = widgets.Layout(display='flex', flex_flow='column', align_items='stretch', border='solid',
                                       width='40%', height='40px')
        self.wbtn_show_team = widgets.Button(description="SHOW TEAMS", disabled=False, layout=button_layout,
                                             visibility="visible", button_style="success")
        self.wbtn_create_team = widgets.Button(description="CREATE FANTASY TEAM", disabled=False,
                                               layout=button_layout,
                                               visibility="visible", button_style="success")
        self.output_team = widgets.Output(layout=widgets.Layout(border='solid', width='100%'))
        self.output_fantasy_team = widgets.Output(layout=widgets.Layout(border='solid', width='100%'))

    def observe_team_widgets(self):
        self.w_lineup.observe(self.set_is_lineup_out, names='value')
        self.w_team1.observe(self.set_team1, names='value')
        self.w_team2.observe(self.set_team2, names='value')
        self.w_optimize.observe(self.set_optimize_on, names='value')
        self.w_min_team1.observe(self.set_min_team1, names='value')
        self.w_max_team1.observe(self.set_max_team1, names='value')
        self.w_top_n_bat.observe(self.set_top_n_bat, names='value')
        self.w_max_bowlers.observe(self.set_max_bowlers, names='value')
        self.w_max_batsmen.observe(self.set_max_batsmen, names='value')
        self.w_max_wk.observe(self.set_max_wk, names='value')
        self.w_max_ar.observe(self.set_max_ar, names='value')
        self.w_spin_weightage.observe(self.set_spin_weightage, names='value')
        self.w_min_credits.observe(self.set_min_credits, names='value')
        self.w_musthave_players.observe(self.set_musthave_players, names='value')
        self.w_exclude_players.observe(self.set_exclude_players, names='value')

        self.wbtn_show_team.on_click(self.on_show_team_details)
        self.wbtn_create_team.on_click(self.on_create_team_clicked)

    def view_team_widgets(self):
        display(widgets.VBox(
            [widgets.Label("ENTER CONFIGURATIONS"), self.w_lineup, widgets.HBox([self.w_team1, self.w_team2]),
             self.w_optimize,
             widgets.HBox([widgets.Label("TEAM 1:"), self.w_min_team1, self.w_max_team1]),
             self.w_top_n_bat,
             widgets.HBox([self.w_max_wk, self.w_max_batsmen, self.w_max_ar, self.w_max_bowlers]),
             self.w_spin_weightage, self.w_min_credits, widgets.HBox([self.w_musthave_players,self.w_exclude_players])]))
        display(widgets.HBox([self.wbtn_create_team, self.wbtn_show_team]))
        display(self.output_team)
        # display(widgets.VBox([self.wbtn_create_team]))
        display(self.output_fantasy_team)

    def set_is_lineup_out(self, b):
        self.CONSIDER_PLAYERS = self.w_lineup.value

    def update_after_team_selection(self):
        self.selected_teams_df = self.data[
            (self.data['Team'] == self.TEAM1) | (self.data['Team'] == self.TEAM2)].reset_index()
        self.players_list = [playername for playername in self.selected_teams_df['PlayerName']]

    def set_team1(self, b):
        self.TEAM1 = self.w_team1.value
        self.update_after_team_selection()
        self.w_musthave_players.options = self.players_list
        self.w_exclude_players.options = self.players_list

    def set_team2(self, b):
        self.TEAM2 = self.w_team2.value
        self.update_after_team_selection()
        self.w_musthave_players.options = self.players_list
        self.w_exclude_players.options = self.players_list

    def set_optimize_on(self, b):
        self.OPTIMIZE_ON = "POINTS"
        if (self.w_optimize.value == "POPULARITY"):
            self.OPTIMIZE_ON = "POPULARITY"
        elif (self.w_optimize.value == "DREAM TEAM COUNT"):
            self.OPTIMIZE_ON = "COUNTDREAMTEAM"
        elif (self.w_optimize.value == "AVG POINTS PER MATCH"):
            self.OPTIMIZE_ON = "AVGPOINTSPERMATCH"
        elif (self.w_optimize.value == "DREAM TEAM COUNT %"):
            self.OPTIMIZE_ON = "COUNTDREAMTEAMPERCENT"

    def set_min_team1(self, b):
        self.MIN_PLAYERS_TEAM1 = self.w_min_team1.value
        self.max_players_list = [count for count in range(self.MIN_PLAYERS_TEAM1, 7 + 1)]

    def set_max_team1(self, b):
        self.MAX_PLAYERS_TEAM1 = self.w_max_team1.value

    def set_top_n_bat(self, b):
        self.CONSIDER_TOP_N_BATSMEN = self.w_top_n_bat.value

    def set_max_bowlers(self, b):
        self.MAX_BOWLERS = self.w_max_bowlers.value

    def set_max_batsmen(self, b):
        self.MAX_BATSMEN = self.w_max_batsmen.value

    def set_max_ar(self, b):
        self.MAX_AR = self.w_max_ar.value

    def set_max_wk(self, b):
        self.MAX_WK = self.w_max_wk.value

    def set_spin_weightage(self, b):
        self.SPIN_WEIGHTAGE_FACTOR = self.w_spin_weightage.value

    def set_min_credits(self, b):
        self.MIN_CREDITS = self.w_min_credits.value

    def set_musthave_players(self, b):
        self.MUST_HAVE_IN_TEAM = list(self.w_musthave_players.value)

    def set_exclude_players(self, b):
        self.EXCLUDE_FROM_TEAM = list(self.w_exclude_players.value)

    def on_create_team_clicked(self, b):
        self.output_fantasy_team.clear_output()
        with self.output_fantasy_team:
            self.create_fantasy_team()

    def on_show_team_details(self, b):
        self.show_team = not self.show_team
        self.output_team.clear_output()
        if self.show_team:
            with self.output_team:
                display(self.selected_teams_df)
                # display(self.data[['PlayerName', 'Team', 'Type', 'Credits', 'BatOrder', 'Points', 'SelectedBy', 'CountDT']])

    ################## WIDGET CONFIGURATIONS - END ########################

    def get_encoded_data(self, input_df):
        # Get type encoded (WK,BAT,AR,BWL)
        from sklearn.preprocessing import OneHotEncoder
        onehotencoder = OneHotEncoder()
        
        # reshape the 1-D country array to 2-D as fit_transform expects 2-D and finally fit the object
        X = onehotencoder.fit_transform(input_df.Type.values.reshape(-1, 1)).toarray()
        # To add this back into the original dataframe
        dfOneHot = pd.DataFrame(X, columns=onehotencoder.get_feature_names(["type"]))
        df = pd.concat([input_df, dfOneHot], axis=1)

        # Update flag for in_team1 players
        # df['in_team1'] = 0
        # df.loc[df['Team'] == self.TEAM1, 'in_team1'] = 1

        df['in_team1'] = df['Team'].apply(lambda x: 1 if x == self.TEAM1 else 0)

        # Update flag for top n batsmen
        # df['is_top_bat'] = 0
        # df.loc[df['BatOrder'] <= self.CONSIDER_TOP_N_BATSMEN, 'is_top_bat'] = 1
        df['is_top_bat'] = df['BatOrder'].apply(lambda x: 1 if x <= self.CONSIDER_TOP_N_BATSMEN else 0)

        # Update weightage factor
        # df['weightage_factor'] = 1
        # df.loc[df['IsSpinner'] == 1, 'weightage_factor'] = self.SPIN_WEIGHTAGE_FACTOR
        df['weightage_factor'] = df['IsSpinner'].apply(lambda x: self.SPIN_WEIGHTAGE_FACTOR if x == 1 else 1)

        # Update must-haves in the team
        df['must_have'] = df['PlayerName'].apply(lambda x: 1 if x in self.MUST_HAVE_IN_TEAM else 0)


        # Update players to exclude from team
        df['exclude'] = df['PlayerName'].apply(lambda x: 1 if x in self.EXCLUDE_FROM_TEAM else 0)

        # Update average points
        df['avg_points'] = round((df['Points'] / df['MatchesPlayed']).replace(np.nan, 0).replace(np.inf, 0), 2)

        # Update average DT Count
        df['countDTPercent'] = round((df['CountDT'] / df['MatchesPlayed']).replace(np.nan, 0).replace(np.inf, 0), 2)

        return df

    def print_output_result(self, fantasy_result_df, summary_df):

        print("######## FANTASY TEAM ############")
        display(fantasy_result_df)

        print("################################")

        print("\n\n######## FANTASY TEAM SUMMARY ############")
        display(summary_df)
        print("################################")

    def create_fantasy_team(self):
        input_team_df = self.get_encoded_data(self.selected_teams_df)
       
        # Set constraints based on the config
        min_players_t1 = self.MIN_PLAYERS_TEAM1 if (
                self.MIN_PLAYERS_TEAM1 > 4 and self.MIN_PLAYERS_TEAM1 <= 7) else 4
        max_players_t1 = self.MAX_PLAYERS_TEAM1 if (
                self.MAX_PLAYERS_TEAM1 < 7 and self.MAX_PLAYERS_TEAM1 >= 4) else 7
        max_bowlers = self.MAX_BOWLERS if (self.MAX_BOWLERS >= 3 and self.MAX_BOWLERS < 6) else 6
        max_batsmen = self.MAX_BATSMEN if (self.MAX_BATSMEN >= 3 and self.MAX_BATSMEN < 6) else 6
        max_wk = self.MAX_WK if (self.MAX_WK >= 1 and self.MAX_WK < 4) else 4
        max_ar = self.MAX_AR if (self.MAX_AR >= 1 and self.MAX_AR < 4) else 4
        min_credits = self.MIN_CREDITS if (self.MIN_CREDITS <= 100) else 0

        # Create the linear programming problem
        prob = plp.LpProblem("FantasyTeamSelectionProblem")

        # Create a list of the players
        players = list(input_team_df['PlayerNumber'])

        # Create a dictionary of credits for all players
        credits = dict(zip(players, input_team_df['Credits']))

        # Create a dictionary for team
        in_team1 = dict(zip(players, input_team_df['in_team1']))

        # Create a dictionary of wk,bat,ar,bwl
        wk_players = dict(zip(players, input_team_df['type_WK']))
        bat_players = dict(zip(players, input_team_df['type_BAT']))
        ar_players = dict(zip(players, input_team_df['type_AR']))
        bwl_players = dict(zip(players, input_team_df['type_BWL']))

        # Create a dictionary based on batting order criteria
        top_bat_players = dict(zip(players, input_team_df['is_top_bat']))

        # Create a dictionary based on must have players
        must_have_players = dict(zip(players, input_team_df['must_have']))


        # Create a dictionary based on exclude players
        exclude_players = dict(zip(players, input_team_df['exclude']))

        # Create a dictionary of player played in last match
        played_last_match = dict(zip(players, input_team_df['PlayedLastMatch']))


        # Create a dictionary of players who have played atleast one match
        played_atleast_one_match = dict(zip(players, input_team_df['PlayedAtleastOneMatch']))

        # Create a dictionary of player in current lineup
        in_lineup = dict(zip(players, input_team_df['InPlaying11']))

        # Create a dictionary of weightage for all players --> to calculate the optimization objective
        weights = dict(zip(players, input_team_df['weightage_factor']))

        # Create a dictionary of points for all players --> maximize on points
        points = dict(zip(players, input_team_df['Points']))

        # Create a dictionary of points for all players --> maximize on popularity of the player
        selectedby = dict(zip(players, input_team_df['Popularity']))

        # Create a dictionary of points for all players --> maximize on count in dream teams
        DTcount = dict(zip(players, input_team_df['CountDT']))

        # Create a dictionary of points for all players --> maximize on average points per match
        avgPoints = dict(zip(players, input_team_df['avg_points']))

        # Create a dictionary of points for all players --> maximize on percentage count in dream teams
        DTcountPercent = dict(zip(players, input_team_df['countDTPercent']))

        # Target variable
        players_vars = plp.LpVariable.dicts("in_fantasy_team", players, cat='Binary')

        # Add main objective function
        if (self.OPTIMIZE_ON == 'POINTS'):
            objective = plp.lpSum([weights[i] * points[i] * players_vars[i] for i in players])
        elif (self.OPTIMIZE_ON == 'POPULARITY'):
            objective = plp.lpSum([weights[i] * selectedby[i] * players_vars[i] for i in players])
        elif (self.OPTIMIZE_ON == 'COUNTDREAMTEAM'):
            objective = plp.lpSum([weights[i] * DTcount[i] * players_vars[i] for i in players])
        elif (self.OPTIMIZE_ON == 'AVGPOINTSPERMATCH'):
            objective = plp.lpSum([weights[i] * avgPoints[i] * players_vars[i] for i in players])
        elif (self.OPTIMIZE_ON == 'COUNTDREAMTEAMPERCENT'):
            objective = plp.lpSum([weights[i] * DTcountPercent[i] * players_vars[i] for i in players])
        else:
            raise Exception(
                f"Incorrect optimization objective. Should be POINTS/POPULARITY/COUNTDREAMTEAM/AVGPOINTSPERMATCH/COUNTDREAMTEAMPERCENT. Value:{self.OPTIMIZE_ON}")

        # Add the constraints
        # Constraint on no. of players - 11
        prob.addConstraint(plp.LpConstraint(e=plp.lpSum([players_vars[f] for f in players]),
                                            sense=plp.LpConstraintEQ, rhs=11, name="constraint_total_count"))

        # Constraint on no. of players from each team - min 4, max 7
        prob.addConstraint(plp.LpConstraint(e=plp.lpSum([in_team1[f] * players_vars[f] for f in players]),
                                            sense=plp.LpConstraintGE, rhs=min_players_t1,
                                            name="constraint_team_count_min"))
        prob.addConstraint(plp.LpConstraint(e=plp.lpSum([in_team1[f] * players_vars[f] for f in players]),
                                            sense=plp.LpConstraintLE, rhs=max_players_t1,
                                            name="constraint_team_count_max"))

        # Constraint on no. of wicketkeepers - min 1, max 3
        prob.addConstraint(plp.LpConstraint(e=plp.lpSum([wk_players[f] * players_vars[f] for f in players]),
                                            sense=plp.LpConstraintGE, rhs=1, name="constraint_wk_low"))
        prob.addConstraint(plp.LpConstraint(e=plp.lpSum([wk_players[f] * players_vars[f] for f in players]),
                                            sense=plp.LpConstraintLE, rhs=max_wk, name="constraint_wk_high"))

        # Constraint on no. of batsmen - min 3, max 6
        prob.addConstraint(plp.LpConstraint(e=plp.lpSum([bat_players[f] * players_vars[f] for f in players]),
                                            sense=plp.LpConstraintGE, rhs=3, name="constraint_bat_low"))
        prob.addConstraint(plp.LpConstraint(e=plp.lpSum([bat_players[f] * players_vars[f] for f in players]),
                                            sense=plp.LpConstraintLE, rhs=max_batsmen, name="constraint_bat_high"))

        # Constraint on no. of batsmen from top order
        prob.addConstraint(plp.LpConstraint(e=plp.lpSum([top_bat_players[f] * players_vars[f] for f in players]),
                                            sense=plp.LpConstraintGE, rhs=3, name="constraint_battop_low"))
        prob.addConstraint(plp.LpConstraint(e=plp.lpSum([top_bat_players[f] * players_vars[f] for f in players]),
                                            sense=plp.LpConstraintLE, rhs=max_batsmen,
                                            name="constraint_battop_high"))

        # Constraint on no. of all-rounders - min 1, max 3
        prob.addConstraint(plp.LpConstraint(e=plp.lpSum([ar_players[f] * players_vars[f] for f in players]),
                                            sense=plp.LpConstraintGE, rhs=1, name="constraint_ar_low"))
        prob.addConstraint(plp.LpConstraint(e=plp.lpSum([ar_players[f] * players_vars[f] for f in players]),
                                            sense=plp.LpConstraintLE, rhs=max_ar, name="constraint_ar_high"))

        # Constraint on no. of bowlers - min 3, max 6
        prob.addConstraint(plp.LpConstraint(e=plp.lpSum([bwl_players[f] * players_vars[f] for f in players]),
                                            sense=plp.LpConstraintGE, rhs=3, name="constraint_bwl_low"))
        prob.addConstraint(plp.LpConstraint(e=plp.lpSum([bwl_players[f] * players_vars[f] for f in players]),
                                            sense=plp.LpConstraintLE, rhs=max_bowlers, name="constraint_bwl_high"))

        # Constraint on total team credits
        prob.addConstraint(plp.LpConstraint(plp.lpSum([credits[f] * players_vars[f] for f in players]),
                                            sense=plp.LpConstraintGE, rhs=min_credits,
                                            name="constraint_credits_low"))
        prob.addConstraint(plp.LpConstraint(plp.lpSum([credits[f] * players_vars[f] for f in players]),
                                            sense=plp.LpConstraintLE, rhs=100, name="constraint_credits_high"))

        # Constraint on must-have players
        prob.addConstraint(plp.LpConstraint(plp.lpSum([must_have_players[f] * players_vars[f] for f in players]),
                                            sense=plp.LpConstraintEQ, rhs=len(self.MUST_HAVE_IN_TEAM),
                                            name="constraint_musthave"))


        # Constraint on exclude players
        prob.addConstraint(plp.LpConstraint(plp.lpSum([exclude_players[f] * players_vars[f] for f in players]),
        sense = plp.LpConstraintEQ, rhs = 0, name = "constraint_exclude"))


        # Add constraints if player is in the lineup for the match or has played the previous match
        if (self.CONSIDER_PLAYERS == 'IN_PLAYING_11'):
            prob.addConstraint(plp.LpConstraint(e=plp.lpSum([in_lineup[f] * players_vars[f] for f in players]),
                                                sense=plp.LpConstraintEQ, rhs=11, name="constraint_in_lineup"))
        elif (self.CONSIDER_PLAYERS == 'PLAYED_LAST_MATCH'):
            prob.addConstraint(
                plp.LpConstraint(e=plp.lpSum([played_last_match[f] * players_vars[f] for f in players]),
                                 sense=plp.LpConstraintEQ, rhs=11, name="constraint_played_last_match"))
        elif (self.CONSIDER_PLAYERS == 'PLAYED_ATLEAST_ONE_MATCH' ):
            prob.addConstraint(
                plp.LpConstraint(e=plp.lpSum([played_atleast_one_match[f] * players_vars[f] for f in players]),
                                 sense=plp.LpConstraintEQ, rhs=11, name="constraint_played_atleast_one_match"))

        # Solve the problem
        prob.sense = plp.LpMaximize
        prob.setObjective(objective)
        prob.solve()
      
        
        # Display the problem representation by PuLP (uncomment to view the results)
        # print("---  Display the problem  - BEGIN ---") 
        #  print(prob) 
        #  print("---  Display the problem  - END ---") 

        
        
       

        in_fantasy_team_list = []
        for i in players:
            in_fantasy_team_list.append(plp.value(players_vars[i]))

        df = self.selected_teams_df
        df['In_Fantasy_Team'] = in_fantasy_team_list

        final_team = df[df['In_Fantasy_Team'] == 1]
        final_team = final_team.sort_values(['Type', 'Credits'])

        # Create output dataframes

        fantasy_result_df = final_team[
            ['PlayerName', 'Team', 'Type', 'Credits', 'BatOrder', 'Points', 'Popularity', 'CountDT',
             'MatchesPlayed']]

        summary_df = pd.DataFrame([], columns=["Details"])
        obj = plp.value(prob.objective)
        summary_df.loc['Solution Status'] = plp.LpStatus[prob.status]
        summary_df.loc['Optimized value'] = round(obj, 2)
        summary_df.loc['Total players points'] = fantasy_result_df['Points'].sum()
        summary_df.loc['Total credit points'] = fantasy_result_df['Credits'].sum()
        summary_df.loc['Wicket-Keepers'] = len(fantasy_result_df[fantasy_result_df['Type'] == 'WK'])
        summary_df.loc['Batsmen'] = len(fantasy_result_df[fantasy_result_df['Type'] == 'BAT'])
        summary_df.loc['All Rounders'] = len(fantasy_result_df[fantasy_result_df['Type'] == 'AR'])
        summary_df.loc['Bowlers'] = len(fantasy_result_df[fantasy_result_df['Type'] == 'BWL'])
        summary_df.loc[f'Team 1 - {self.TEAM1}'] = len(fantasy_result_df[fantasy_result_df['Team'] == self.TEAM1])
        summary_df.loc[f'Team 2 - {self.TEAM2}'] = len(fantasy_result_df[fantasy_result_df['Team'] == self.TEAM2])


        if self.print_result:
            self.print_output_result(fantasy_result_df,summary_df) 
            
        final_result = {"Fantasy Team": fantasy_result_df, "Summary": summary_df}
       
        return final_result
        




In [100]:
parameters = {'TEAM1':'RR','TEAM2':'DC','OPTIMIZE_ON':'POINTS','EXCLUDE_FROM_TEAM':['Rishabh Pant']}

fantasy_team = FantasyTeamGenerator(with_widgets=True, config_params=parameters,
                                    print_result=True,
                                    output_folder = None)
team_df = fantasy_team.create_fantasy_team()



Output(layout=Layout(border='solid', width='100%'))

Output(layout=Layout(border='solid', width='100%'))

######## FANTASY TEAM ############


,PlayerName,Team,Type,Credits,BatOrder,Points,Popularity,CountDT,MatchesPlayed
1,R Tewatia,RR,AR,8.5,6,264.0,60.0,3,5
3,Axar Patel,DC,AR,8.5,7,155.0,9.0,4,4
2,Marcus Stoinis,DC,AR,9.0,5,259.0,61.0,2,5
14,Prithvi Shaw,DC,BAT,8.5,1,264.0,7.0,4,5
12,Shreyas Iyer,DC,BAT,9.5,4,254.0,86.0,3,5
10,S Smith,RR,BAT,10.0,2,194.0,85.0,3,5
24,T Curran,RR,BWL,8.5,7,183.0,37.0,3,5
30,Anrich Nortje,DC,BWL,8.5,10,219.0,54.0,3,5
22,J Archer,RR,BWL,9.0,9,263.0,84.0,4,5
21,Kagiso Rabada,DC,BWL,9.5,9,368.0,75.0,4,5


################################


######## FANTASY TEAM SUMMARY ############


,Details
Solution Status,Optimal
Optimized value,2731
Total players points,2731
Total credit points,99
Wicket-Keepers,1
Batsmen,3
All Rounders,3
Bowlers,4
Team 1 - RR,5
Team 2 - DC,6


################################


In [ ]:
## Additional code to print the logs from the PuLP solver

In [ ]:
# Return the pulp problem variable from create_fantasy_team() to view the logs

# myprob = fantasy_team.create_fantasy_team()

In [87]:
import pickle
with open("./file_temp.pk", 'wb') as handle:
    pickle.dump(myprob, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [88]:
%%python

import pickle
import pulp
import pandas as pd

with open("./file_temp.pk", 'rb') as handle:

    prob = pickle.load(handle)

    #Optimization
    prob.solve(pulp.PULP_CBC_CMD(msg=True))

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/le0080/.pyenv/versions/3.6.6/lib/python3.6/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/c9/mj152xms3x59590s1c16yt85j6k44m/T/b198e8bfe428461e8fe5380a2efabf8c-pulp.mps max ratio None allow None threads None presolve on strong None gomory on knapsack on probing on branch printingOptions all solution /var/folders/c9/mj152xms3x59590s1c16yt85j6k44m/T/b198e8bfe428461e8fe5380a2efabf8c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 22 COLUMNS
At line 446 RHS
At line 464 BOUNDS
At line 511 ENDATA
Problem MODEL has 17 rows, 46 columns and 303 elements
Coin0008I MODEL read with 0 errors
String of None is illegal for double parameter ratioGap value remains 0
String of None is illegal for double parameter allowableGap value remains 0
String of None is illegal for integer parameter threads value remains 0
String of None is illegal for integer paramete